In [1]:
# === P-LVAL — Factor vs monto esperado (percentiles para evaluar) ==============
# Regla: Cash transaction > {var.Factor} × (largest expected amount del cliente).
# Justificación original: factor por transacción = tx_base_amount / expected_max;
#   redondear cada factor al ENTERO más cercano; tomar la media de esos enteros,
#   y elegir el SIGUIENTE número natural tras esa media.
# Aquí, además, mostramos percentiles (p90/p95/p97/p99) tanto de factor RAW como del ENTERO.

import pandas as pd
import numpy as np
import math

# -------- Parámetros editables --------
PATH = "../../data/tx/datos_trx__with_subsub.csv"
SUBSUBSEGMENTS = "R-Low"   # <-- ajusta el sub-subsegmento
PCTS = [90, 95, 97, 99]

# -------- Carga mínima --------
df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# -------- Preparación --------
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["customer_expected_amount"] = pd.to_numeric(df["customer_expected_amount"], errors="coerce")

# Solo Cash y datos válidos
is_cash = (df["tx_type"].astype(str).str.title() == "Cash")
m = df.loc[is_cash, ["customer_id", "tx_base_amount", "customer_expected_amount"]].dropna()

if m.empty:
    print("No hay transacciones Cash con datos suficientes.")
else:
    # largest expected amount por cliente
    exp_by_cust = (
        m.groupby("customer_id", as_index=False)["customer_expected_amount"].max()
        .rename(columns={"customer_expected_amount": "expected_max"})
    )

    m = m.merge(exp_by_cust, on="customer_id", how="left")
    m = m[(m["expected_max"] > 0) & (m["tx_base_amount"] > 0)].copy()

    if m.empty:
        print("No hay pares (tx_base_amount, expected_max) válidos para calcular factores.")
    else:
        # Factor RAW por transacción
        m["factor_raw"] = m["tx_base_amount"] / m["expected_max"]

        # Factor ENTERO por transacción (redondeo al entero más cercano)
        m["factor_int"] = np.rint(m["factor_raw"])

        # Percentiles
        s_raw = m["factor_raw"].astype(float).replace([np.inf, -np.inf], np.nan).dropna()
        s_int = m["factor_int"].astype(float).replace([np.inf, -np.inf], np.nan).dropna()

        stats_raw = {f"p{p}": float(np.percentile(s_raw, p)) for p in PCTS} if len(s_raw) else {}
        stats_int = {f"p{p}": float(np.percentile(s_int, p)) for p in PCTS} if len(s_int) else {}

        # Sugerencia (según tu justificación): siguiente natural tras la media de enteros
        mean_int = float(s_int.mean()) if len(s_int) else np.nan
        recommended = int(np.floor(mean_int)) + 1 if np.isfinite(mean_int) else np.nan

        print("=== P-LVAL — Factor vs 'largest expected amount' (Cash) ===")
        print(f"Transacciones consideradas: {len(m)}")
        print("\n-- Percentiles (RAW) --")
        if stats_raw:
            for p in PCTS:
                v = stats_raw[f"p{p}"]
                print(f"p{p:>2}: {v:,.2f}")
        else:
            print("Sin datos.")

        print("\n-- Percentiles (ENTERO redondeado) --")
        if stats_int:
            for p in PCTS:
                v = stats_int[f"p{p}"]
                print(f"p{p:>2}: {v:,.2f}")
        else:
            print("Sin datos.")

        print(f"\nMedia de enteros: {mean_int:,.2f}" if np.isfinite(mean_int) else "\nMedia de enteros: NA")
        print(f"Sugerencia (siguiente natural tras la media de enteros): {recommended}")


C:\Users\vinso\AppData\Local\Temp\ipykernel_26128\2433122614.py:18: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")


=== P-LVAL — Factor vs 'largest expected amount' (Cash) ===
Transacciones consideradas: 176586

-- Percentiles (RAW) --
p90: 0.08
p95: 0.15
p97: 0.24
p99: 0.63

-- Percentiles (ENTERO redondeado) --
p90: 0.00
p95: 0.00
p97: 0.00
p99: 1.00

Media de enteros: 0.02
Sugerencia (siguiente natural tras la media de enteros): 1


# Simulación alertas

In [3]:
# === P-LVAL — Sensibilidad (Actual vs propuestos) ==============================
# LÓGICA EXACTA:
# customer_expected_amount [Default: 0] ≠ 0
# AND tx_base_amount > customer_expected_amount * [Factor]
# Unidad = transacciones que cumplen (IN/OUT Cash; puedes ajustar tipo si prefieres)

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = ["I-2"]                # <-- ajusta el sub-subsegmento
PARAMS={
    "Actual":{"Factor":4.36},
    "p95":   {"Factor":2},
    "p97":   {"Factor":7},
    "p99":   {"Factor":26},
}

df=pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_base_amount"]=pd.to_numeric(df["tx_base_amount"], errors="coerce")
exp=pd.to_numeric(df["customer_expected_amount"], errors="coerce").fillna(0)
df["_exp"]=exp

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

g=df[df["tx_base_amount"].notna()].copy()

order=["Actual","p90","p95","p97","p99"]
param_tbl=(pd.DataFrame(PARAMS).T.loc[[k for k in order if k in PARAMS]]
           .rename_axis("escenario").reset_index())
print("=== P-LVAL — Parámetros ==="); display(param_tbl)

counts={}
base_elig=(g["_exp"]!=0)
for k,v in PARAMS.items():
    F=v["Factor"]
    counts[k]=int((base_elig & (g["tx_base_amount"] > g["_exp"]*F)).sum())

out=pd.DataFrame([{
    "alertas_actual":counts.get("Actual",0),
    "alertas_p90":counts.get("p90",0),
    "alertas_p95":counts.get("p95",0),
    "alertas_p97":counts.get("p97",0),
    "alertas_p99":counts.get("p99",0),
}])
print("=== P-LVAL — Alertas por escenario (tx) ==="); display(out)


C:\Users\vinso\AppData\Local\Temp\ipykernel_22556\1559886711.py:19: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-LVAL — Parámetros ===


,escenario,Factor
0,Actual,4
1,p95,2
2,p97,7
3,p99,26


=== P-LVAL — Alertas por escenario (tx) ===


,alertas_actual,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,103,0,143,75,25
